# Crypto Information Bedrcok Guardrails Demo


Guardrails for Amazon Bedrock evaluates user inputs and FM responses based on use case specific policies, and provides an additional layer of safeguards regardless of the underlying FM. Guardrails can be applied across all large language models (LLMs) on Amazon Bedrock, including fine-tuned models. Customers can create multiple guardrails, each configured with a different combination of controls, and use these guardrails across different applications and use cases. 

This notebook showcases the implementation of AWS Bedrock guardrails for handling financial investment related queries.  


## Prerequisites:
#### SageMaker Access:
* Access to AWS SageMaker Console
* Permissions to create/use SageMaker notebooks
* Instance Requirements:

#### A running SageMaker notebook instance

#### Default Python 3 kernel

#### IAM Role Requirements:

The SageMaker execution role needs these permissions, Note: use a more fine grained policy for better security:

* bedrock:* 
* bedrock-runtime:*

Region Configuration:
* Ensure your notebook is in a region where Bedrock and Guardrails are available
  
That's it!


If the next two cells run without errors, you're ready to proceed with the demo.


In [ ]:
#Start by installing the dependencies to ensure a recent version
!pip install --upgrade --force-reinstall boto3
import boto3
print(boto3.__version__)

In [37]:
client = boto3.client('bedrock')

#### Creating a Guardrail

In [38]:
# Create a guardrail for a crypto advisor app

create_response = client.create_guardrail(
    name='fintech-chatbot-guardian2',
    description='Create guardrails for an educational financial app',
    topicPolicyConfig={
        'topicsConfig': [
        {
            'name': 'SpecificInvestmentAdvice',
            'definition': 'Specific financial recommendations, investment timing advice, or price predictions for cryptocurrencies',
            'examples': [
                'What  should I invest in right now?',
                            
            ],
            'type': 'DENY'
        }
    ]
}
            
       
    ,
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {
                'text': 'crypto recommendations'
            },
            {
                'text': 'crypto advice'
            },
            {
                'text': 'trading signals'
            },
            {
                'text': 'price prediction'
            },
        ],
        'managedWordListsConfig': [
            {
                'type': 'PROFANITY'
            }
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {
                'type': 'EMAIL',
                'action': 'ANONYMIZE'
            },
            {
                'type': 'PHONE',
                'action': 'ANONYMIZE'
            },
            {
                'type': 'NAME',
                'action': 'ANONYMIZE'
            },
            {
                'type': 'US_SOCIAL_SECURITY_NUMBER',
                'action': 'BLOCK'
            },
            {
                'type': 'US_BANK_ACCOUNT_NUMBER',
                'action': 'BLOCK'
            },
            {
                'type': 'CREDIT_DEBIT_CARD_NUMBER',
                'action': 'BLOCK'
            }
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    blockedInputMessaging='I apologize, but I cannot provide investment advice, trading recommendations, or price predictions for cryptocurrencies. However, I can help you understand how blockchain technology works or provide general educational information.',
    blockedOutputsMessaging='I apologize, but I cannot provide investment advice, trading recommendations, or price predictions for cryptocurrencies. However, I can help you understand how blockchain technology works or provide general educational information.',
)

print(create_response)

{'ResponseMetadata': {'RequestId': 'b8f19273-9b14-4f4d-8bb4-50573fa51f6e', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Thu, 20 Feb 2025 19:10:17 GMT', 'content-type': 'application/json', 'content-length': '172', 'connection': 'keep-alive', 'x-amzn-requestid': 'b8f19273-9b14-4f4d-8bb4-50573fa51f6e'}, 'RetryAttempts': 0}, 'guardrailId': 'e9moyhumurn9', 'guardrailArn': 'arn:aws:bedrock:us-west-2:352768194846:guardrail/e9moyhumurn9', 'version': 'DRAFT', 'createdAt': datetime.datetime(2025, 2, 20, 19, 10, 17, 772459, tzinfo=tzlocal())}


### Testing the Guardrail

In [33]:
import boto3
import json

# Initialize the Bedrock client
bedrock = boto3.client('bedrock-runtime')

# get Guardrail ID
guardrail_id = create_response['guardrailId']


"""  Processes user input through Amazon Bedrock's Claude model with applied guardrails.
    
    Args:
        input_text (str): The user's input text to be processed
        
    Returns:
        dict: A dictionary containing the 'completion' key with either:
            - The model's guardrailed response as a string
            - An error message if the processing fails
"""
def process_with_guardrail(input_text):
    try:
        response = bedrock.invoke_model_with_response_stream(
            modelId='anthropic.claude-v2',
            contentType='application/json',
            accept='application/json',
            body=json.dumps({
                'prompt': f"\n\nHuman: {input_text}\n\nAssistant:",
                'max_tokens_to_sample': 300,
                'temperature': 0.5,
                'top_k': 250,
                'top_p': 0.9,
                'stop_sequences': ['\n\nHuman:'],
                'anthropic_version': 'bedrock-2023-05-31'
            }),
            guardrailIdentifier=guardrail_id,
            guardrailVersion='DRAFT'
        )
        
        # Process the streaming response
        response_body = []
        for event in response['body']:
            chunk = json.loads(event['chunk']['bytes'].decode())
            response_body.append(chunk.get('completion', ''))
            
        return {"completion": ''.join(response_body)}
        
    except Exception as e:
        return {"completion": f"Error processing request: {str(e)}"}

# Test the guardrail with some sample inputs
sample_inputs = [
    "What is the blockchain?", 
    'What is a 401k?',
    "Should I invest in Bitcoin?",
    "Here's my credit card number: 1234-5678-9012-3456",
    "Is cryptocurrency a good investment?",
    "Will Bitcoin price reach 100000?", 
    
    ]

for input_text in sample_inputs:
    print(f"\nINPUT: {input_text}")
    result = process_with_guardrail(input_text)
    print(f"RESPONSE: {result['completion']}")


INPUT: What is the blockchain?
RESPONSE:  The blockchain is a distributed digital ledger technology that serves as the foundation for cryptocurrencies like Bitcoin and records transactions in a secure and transparent manner across a decentralized network. Some key features of the blockchain include:

- Decentralized - The blockchain ledger is distributed across a network of computers rather than being stored on a central server. This makes it more resistant to hacking or manipulation.

- Transparent - Transactions on the blockchain are visible to everyone participating in the network. While individual user identities may be anonymous, the transactions themselves are public. 

- Secure - Cryptography like hashing and digital signatures are used to secure transactions on the blockchain. This makes it very difficult to tamper with or forge transactions.

- Immutable - Once data is recorded on the blockchain, it is extremely difficult to alter it retroactively. This provides a permanent a

#### Getting a Guardrail, creating a version and listing all the versions and Drafts

In [41]:
# Getthe DRAFT version 
get_response = client.get_guardrail(
    guardrailIdentifier=create_response['guardrailId'],
    guardrailVersion='DRAFT'
)


In [42]:
# Get the published version (Version 1) from the DRAFT
version_response = client.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of Guardrail that has HIGH content filters across'
)

In [43]:
#Lists all versions of the c guardrail up to 3 versions. includes the DRAFT version
list_guardrails_response = client.list_guardrails(
    guardrailIdentifier=create_response['guardrailArn'],
    maxResults=5)

print(list_guardrails_response)

{'ResponseMetadata': {'RequestId': '879394b1-3bcb-4f71-aaa8-d3442703efd9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 20 Feb 2025 19:16:33 GMT', 'content-type': 'application/json', 'content-length': '654', 'connection': 'keep-alive', 'x-amzn-requestid': '879394b1-3bcb-4f71-aaa8-d3442703efd9'}, 'RetryAttempts': 0}, 'guardrails': [{'id': 'e9moyhumurn9', 'arn': 'arn:aws:bedrock:us-west-2:352768194846:guardrail/e9moyhumurn9', 'status': 'READY', 'name': 'fintech-chatbot-guardian1', 'description': 'Create guardrails for an educational financial app', 'version': 'DRAFT', 'createdAt': datetime.datetime(2025, 2, 20, 19, 10, 17, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 2, 20, 19, 16, 31, 548594, tzinfo=tzlocal())}, {'id': 'e9moyhumurn9', 'arn': 'arn:aws:bedrock:us-west-2:352768194846:guardrail/e9moyhumurn9', 'status': 'READY', 'name': 'fintech-chatbot-guardian1', 'description': 'Version of Guardrail that has HIGH content filters across', 'version': '1', 'createdAt': date

In [40]:
print("The End")

The End
